# LeTour data set 
This file downloads raw data about every rider of every Tour de France (from 1903 up to 2020). This data will then be postprocessed and stored in CSV format.
Executing this notebook might take some minutes.

## 1) Retrieve urls for data extract
First we generate the urls that we need to download the raw HTML pages from the `letour.fr` website to work offline from here on. The dataframe dflink will stores the respective url for each year.
Take a look at `view-source:https://www.letour.fr/en/history` (at around line 1143-1890).  

In [1]:
import os
import subprocess
import collections
import re
from pathlib import Path
import datetime
import time

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [3]:
PREFIX = 'http://www.letour.fr'
HISTORYPAGE = 'https://www.letour.fr/en/history'


In [4]:
headers = {'Accept': 'text/html', 'User-Agent': 'python-requests/1.2.0','Accept-Charset': 'utf-8','accept-encoding': 'deflate, br'}

In [5]:
resulthistpage = requests.get(HISTORYPAGE, allow_redirects=True, headers=headers)
souphistory = BeautifulSoup(resulthistpage.text, 'html.parser')

In [9]:
# Find select tag for histo links
select_tag_histo = souphistory.find_all("button",{"class" :"dateTabs__link"})

LH = [x["data-tabs-ajax"] for x in select_tag_histo]

dflink = pd.DataFrame({'TDFHistorylink':LH})
dflink

,TDFHistorylink
0,/en/block/history/10707/d0ab6a216569236433268b...
1,/en/block/history/10708/0b76b8f809ad5d8bcf3579...
2,/en/block/history/10709/c5f53ced72a23f333cc186...
3,/en/block/history/10710/ead1d1704b1600c795619e...
4,/en/block/history/10711/f558d3bc819c8ee5dc627d...
...,...
104,/en/block/history/11818/f34c3404d95a697dcf77d4...
105,/en/block/history/11819/96c0eb3fa403ebf222f28b...
106,/en/block/history/11820/17fa8e795e69e9f326ea26...
107,/en/block/history/11821/a3fe7d350ef546e3c27ae8...


## 2) Get data from HTML pages and convert results to dataframes

### 2.1) Create function that convert HHh mm' ss'' to seconds

In [7]:
def calcTotalSeconds(row,mode):

   val = sum(x * int(t) for x, t in zip([3600, 60, 1], row.replace("h",":").replace("'",":").replace('"',':').replace(" ","").replace("+","").replace("-","0").split(":")))
   
   if ((mode=='Gap') and val > 180000) :
        val=0
    
   return val

### 2.2) Create a function that will retrieve elements from a source HTML page located on an input url

In [8]:
def getstagesNrank(i_url):
    resultfull = requests.get(i_url, allow_redirects=True)
    result = resultfull.text
    resultstatus = resultfull.status_code
    
    print(i_url + ' ==> HTTP STATUS = ' + str(resultstatus))
    
    soup = BeautifulSoup(result, 'html.parser')  
    h=soup.find('h3')
    year=int(h.text[-4:])

    # Find select tag
    select_tag = soup.find("select")

    # find all option tag inside select tag
    options = select_tag.find_all("option")

    cols = ['Year','TotalTDFDistance','Stage']
    lst = []
    
    #search for stages
    distance=soup.select("[class~=statsInfos__number]")[1].contents
               
    #search for distance of the TDF edition
    for option in options:
       lst.append([year,int(distance[0].replace(" ","")),option.text])
               
    dfstages = pd.DataFrame(lst, columns=cols)
    
    
    # Find select tag for ranking racers
    rankingTable = soup.find("table")

    dfrank = pd.read_html(str(rankingTable))[0]

    dfrank['Year']=year
    dfrank['Distance (km)']=int(distance[0].replace(" ",""))
    dfrank['Number of stages']=len(dfstages)
    dfrank['TotalSeconds']=dfrank['Times'].apply(lambda x:calcTotalSeconds(x,'Total'))
    dfrank['GapSeconds']=dfrank['Gap'].apply(lambda x:calcTotalSeconds(x,'Gap'))

    
    return dfstages, dfrank

### 2.3) Loop on the dflink dataframe to get data from each url source

In [12]:
dfstagesres=[]
dfstagesrestmp=[]
dfrankres=[]
dfrankrestmp=[]
dfrankoutput=[]

for index, row  in dflink.iterrows():
    url = PREFIX+row['TDFHistorylink']
    try :
      if index >= 12 :  # limit from 1919 (data need to be cleaned a little bit more before that)
        dfstagesres, dfrankres=getstagesNrank(url)
        dfstagesrestmp= dfstagesres.append(dfstagesrestmp, ignore_index=True)  
        dfrankrestmp=dfrankres.append(dfrankrestmp, ignore_index=True)
        
        
    except:
      raise

    
dfstageoutput=dfstagesrestmp
dfrankoutput=dfrankrestmp

http://www.letour.fr/en/block/history/10719/f15cf81b7599ea113d2b1d614f73bf83 ==> HTTP STATUS = 200


C:\Users\romain\AppData\Local\Temp\ipykernel_15988\2670945799.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfstagesrestmp= dfstagesres.append(dfstagesrestmp, ignore_index=True)
C:\Users\romain\AppData\Local\Temp\ipykernel_15988\2670945799.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfrankrestmp=dfrankres.append(dfrankrestmp, ignore_index=True)


http://www.letour.fr/en/block/history/10720/53ac40095834ae1aae5d197aa730799a ==> HTTP STATUS = 200


C:\Users\romain\AppData\Local\Temp\ipykernel_15988\2670945799.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfstagesrestmp= dfstagesres.append(dfstagesrestmp, ignore_index=True)
C:\Users\romain\AppData\Local\Temp\ipykernel_15988\2670945799.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfrankrestmp=dfrankres.append(dfrankrestmp, ignore_index=True)


http://www.letour.fr/en/block/history/10721/c663ba39ac60d58c55a1a787a23025fc ==> HTTP STATUS = 200


...

C:\Users\romain\AppData\Local\Temp\ipykernel_15988\2670945799.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfstagesrestmp= dfstagesres.append(dfstagesrestmp, ignore_index=True)
C:\Users\romain\AppData\Local\Temp\ipykernel_15988\2670945799.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfrankrestmp=dfrankres.append(dfrankrestmp, ignore_index=True)


http://www.letour.fr/en/block/history/11821/a3fe7d350ef546e3c27ae83a84fa3a0b ==> HTTP STATUS = 200


C:\Users\romain\AppData\Local\Temp\ipykernel_15988\2670945799.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfstagesrestmp= dfstagesres.append(dfstagesrestmp, ignore_index=True)
C:\Users\romain\AppData\Local\Temp\ipykernel_15988\2670945799.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfrankrestmp=dfrankres.append(dfrankrestmp, ignore_index=True)


http://www.letour.fr/en/block/history/11822/ca646e1b4b99d7ee973c457c87392395 ==> HTTP STATUS = 200


C:\Users\romain\AppData\Local\Temp\ipykernel_15988\2670945799.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfstagesrestmp= dfstagesres.append(dfstagesrestmp, ignore_index=True)
C:\Users\romain\AppData\Local\Temp\ipykernel_15988\2670945799.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfrankrestmp=dfrankres.append(dfrankrestmp, ignore_index=True)


In [13]:
dfrankoutput

,Rank,Rider,Rider No.,Team,Times,Gap,B,P,Year,Distance (km),Number of stages,TotalSeconds,GapSeconds
0,1,JONAS VINGEGAARD,18,JUMBO - VISMA,79h 33' 20'',-,32',NaN,2022,3328,22,286400,0
1,2,TADEJ POGAČAR,1,UAE TEAM EMIRATES,79h 36' 03'',+ 00h 02' 43'',40',NaN,2022,3328,22,286563,163
2,3,GERAINT THOMAS,21,INEOS GRENADIERS,79h 40' 42'',+ 00h 07' 22'',NaN,NaN,2022,3328,22,286842,442
3,4,DAVID GAUDU,91,GROUPAMA - FDJ,79h 46' 59'',+ 00h 13' 39'',04',NaN,2022,3328,22,287219,819
4,5,ALEKSANDR VLASOV,41,BORA - HANSGROHE,79h 49' 06'',+ 00h 15' 46'',NaN,NaN,2022,3328,22,287346,946
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9338,6,JACQUES COOMANS,63,TDF 1919 ***,246h 28' 49'',+ 15h 21' 34'',NaN,NaN,1919,5560,15,887329,55294
9339,7,LUIGI LUCOTTI,19,TDF 1919 ***,247h 08' 27'',+ 16h 01' 12'',NaN,NaN,1919,5560,15,889707,57672
9340,8,JOSEPH VAN DAELE,55,TDF 1919 ***,249h 30' 17'',+ 18h 23' 02'',NaN,NaN,1919,5560,15,898217,66182
9341,9,ALFRED STEUX,56,TDF 1919 ***,251h 36' 16'',+ 20h 29' 01'',NaN,NaN,1919,5560,15,905776,73741


## 3) Clean up the data 


In [14]:
# Fix result types
dfrankoutput["ResultType"] = "time"
dfrankoutput.loc[dfrankoutput["Year"].isin([1905,1906,1908]),"ResultType"] = "null"
dfrankoutput.loc[dfrankoutput["Year"].isin([1907,1909,1910,1911,1912]),"ResultType"] = "points"

In [15]:
# Fix this weird bug for e.g. year 2006 and 1997
for year in np.unique(dfrankoutput["Year"]):
    tmp = dfrankoutput[dfrankoutput["Year"]==year].reset_index()
    if tmp.loc[0]["TotalSeconds"] > tmp.loc[2]["TotalSeconds"]:
        print(year)
# Okay seems to be only for 2006 and 1997


1997
2006


In [16]:
tmp = dfrankoutput[dfrankoutput["Year"]==2006].reset_index()
ts = np.array(tmp["TotalSeconds"])
gs = np.array(tmp["GapSeconds"])
ts[1:] = ts[0]+gs[1:]

dfrankoutput.loc[dfrankoutput["Year"]==2006,"TotalSeconds"] = ts

tmp = dfrankoutput[dfrankoutput["Year"]==1997].reset_index()
ts = np.array(tmp["TotalSeconds"])
gs = np.array(tmp["GapSeconds"])
ts[1:] = ts[0]+gs[1:]

dfrankoutput.loc[dfrankoutput["Year"]==1997,"TotalSeconds"] = ts

## 4) Write Data to CSV

In [17]:
dfrankoutput.to_csv("../data/TDF_Riders_History.csv")

In [18]:
dfstageoutput.to_csv("../data/TDF_Stages_History.csv")